# Test Base Input Data

ほっといてもなかなかデータ出来ないのでとりあえず叩き台のインプットデータ作成

とりあえず
* ドル円為替（終値)
* 日経平均（終値)
* S&P500（終値, Volume, 修正終値）
* JGB10Y
* VIX（終値)
だけ用意.

またこれらに対して欠損日は線形補間とし、
* 絶対額
* 前日比
* 30日移動平均
* 120日移動平均
を暫定的に追加.

正規化はしていないので、インプット前に実施する必要あり.

In [1]:
import quandl as ql
import pandas as pd

In [2]:
ql.ApiConfig.api_key = 'CbzNMfBFSFEFsiNNAe97' # APIKeyを置かないと取得制限

In [7]:
# 日本国債金利(10年)
df_Interest_JP = ql.get('MOFJ/INTEREST_RATE_JAPAN_10Y', start_date='2001-01-01')
df_Interest_JP.columns = ["JGB10"]
df_Interest_JP.head(5)

,Value
Date,
2001-01-04,1.639
2001-01-05,1.630
2001-01-09,1.565
2001-01-10,1.581
2001-01-11,1.519


In [33]:
# USD/JPY
df_JPYUSD = ql.get('FRED/DEXJPUS', start_date='2001-01-01')
df_JPYUSD.columns = ["JP/US"]
df_JPYUSD.head(5)

,JP/US
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


In [23]:
# 日経平均株価
df_STOCKJP = ql.get('NIKKEI/INDEX'+".4", start_date='2001-01-01')
df_STOCKJP.columns = ["Nikkei/Close"]

In [29]:
# S&P株価　終値、ボリューム、修正後終値
df_STOCKSP = ql.get('YAHOO/INDEX_GSPC', start_date="2001-01-01")
df_STOCKSP = df_STOCKSP[["Close","Volume","Adjusted Close"]]
df_STOCKSP.columns = ["S&P/Close","S&P/Volume","S&P/AdjClose"]
df_STOCKSP.head(10)

,S&P/Close,S&P/Volume,S&P/AdjClose
Date,,,
2001-01-02,1283.270020,1129400000,1283.270020
2001-01-03,1347.560059,1880700000,1347.560059
2001-01-04,1333.339966,2131000000,1333.339966
2001-01-05,1298.349976,1430800000,1298.349976
2001-01-08,1295.859985,1115500000,1295.859985
2001-01-09,1300.800049,1191300000,1300.800049
2001-01-10,1313.270020,1296500000,1313.270020
2001-01-11,1326.819946,1411200000,1326.819946
2001-01-12,1318.550049,1276000000,1318.550049


In [32]:
# VIX
df_VIX = ql.get('NIKKEI/INDEX'+".4", start_date='2001-01-01')
df_VIX.columns = ["VIX/Close"]
df_VIX.tail(10)

,VIX/Close
Date,
2016-06-21,16169.11
2016-06-22,16065.72
2016-06-23,16238.35
2016-06-24,14952.02
2016-06-27,15309.21
2016-06-28,15323.14
2016-06-29,15566.83
2016-06-30,15575.92
2016-07-01,15682.48


In [48]:
#横方向への結合, 'inner'で存在する箇所のみ
merge_list = [df_JPYUSD, df_Interest_JP, df_STOCKJP, df_STOCKSP, df_VIX]
df_merge = pd.concat(merge_list, axis=1)#, join='inner')

In [54]:
# 欠損処理など
Data=df_merge.to_period(freq="D") # Dayをタイムピリオドに設定
Data = Data.interpolate() # データの前後で線形補間(データの前後以外nanはなくなる)
Data = Data.dropna(axis=0) # nanの行を落とす

In [61]:
# 前期比、移動平均
DataLag = Data.pct_change() # 前期比
DataLag.columns = [
    "JP/US.Lag", "JGB10.Lag", "Nikkei/Close.Lag", "S&P/Close.Lag" , "S&P/Volume.Lag", "S&P/AdjClose.Lag", "VIX/Close.Lag"]

DataMoveAve30 = pd.rolling_mean(Data,30) # 30day移動平均
DataMoveAve30.columns = [
    "JP/US.MA30", "JGB10.MA30", "Nikkei/Close.MA30", "S&P/Close.MA30" , "S&P/Volume.MA30", "S&P/AdjClose.MA30", "VIX/Close.MA30"]

DataMoveAve120 = pd.rolling_mean(Data,120) # 120day移動平均
DataMoveAve120.columns = [
    "JP/US.MA120", "JGB10.MA120", "Nikkei/Close.MA120", "S&P/Close.MA120" , "S&P/Volume.MA120", "S&P/AdjClose.MA120", "VIX/Close.MA120"]

In [73]:
InputList = [Data, DataLag, DataMoveAve30, DataMoveAve120 ]
InputData = pd.concat(InputList, axis=1)
InputData = InputData.dropna(axis=0) # nanの行を落とす

In [75]:
InputData.head(20)

,JP/US,JGB10,Nikkei/Close,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close,JP/US.Lag,JGB10.Lag,Nikkei/Close.Lag,...,S&P/Volume.MA30,S&P/AdjClose.MA30,VIX/Close.MA30,JP/US.MA120,JGB10.MA120,Nikkei/Close.MA120,S&P/Close.MA120,S&P/Volume.MA120,S&P/AdjClose.MA120,VIX/Close.MA120
2001-06-20,123.300,1.201,12674.64,1223.140015,1350100000,1223.140015,12674.64,0.002684,0.029135,0.007983,...,1.102100e+09,1261.507666,13461.619000,120.279083,1.321554,13386.141000,1254.843961,1.200070e+09,1254.843961,13386.141000
2001-06-21,124.400,1.182,12962.43,1237.040039,1546820000,1237.040039,12962.43,0.008921,-0.015820,0.022706,...,1.118437e+09,1260.902999,13426.440333,120.353500,1.317746,13380.065500,1254.041461,1.195202e+09,1254.041461,13380.065500
2001-06-22,124.250,1.171,13044.61,1225.349976,1189200000,1225.349976,13044.61,-0.001206,-0.009306,0.006340,...,1.127871e+09,1260.225663,13393.130000,120.420667,1.313921,13373.207167,1253.433128,1.193188e+09,1253.433128,13373.207167
2001-06-25,124.020,1.144,12896.47,1218.599976,1050100000,1218.599976,12896.47,-0.001851,-0.023057,-0.011356,...,1.134267e+09,1259.214994,13360.578333,120.487750,1.310142,13366.185583,1252.789294,1.192644e+09,1252.789294,13366.185583
2001-06-26,123.820,1.140,12978.82,1216.760010,1198900000,1216.760010,12978.82,-0.001613,-0.003497,0.006385,...,1.138504e+09,1258.125663,13324.738000,120.547583,1.306600,13360.921500,1252.088961,1.192707e+09,1252.088961,13360.921500
2001-06-27,124.250,1.133,12828.98,1211.069946,1162100000,1211.069946,12828.98,0.003473,-0.006140,-0.011545,...,1.130397e+09,1255.661662,13295.895000,120.614167,1.302867,13355.890917,1251.237293,1.191587e+09,1251.237293,13355.890917
2001-06-28,124.630,1.125,12679.88,1226.199951,1327300000,1226.199951,12679.88,0.003058,-0.007061,-0.011622,...,1.129454e+09,1253.585327,13254.868667,120.673083,1.299583,13351.547667,1250.398794,1.190888e+09,1250.398794,13351.547667
2001-06-29,124.730,1.183,12969.05,1224.380005,1832360000,1224.380005,12969.05,0.000802,0.051556,0.022805,...,1.152839e+09,1251.332662,13224.578000,120.731167,1.297108,13348.391917,1249.614043,1.195524e+09,1249.614043,13348.391917
2001-07-02,124.220,1.263,12751.18,1236.719971,1128300000,1236.719971,12751.18,-0.004089,0.067625,-0.016799,...,1.151286e+09,1248.795663,13177.056333,120.787000,1.295142,13342.099833,1248.898376,1.194586e+09,1248.898376,13342.099833
2001-07-03,124.480,1.272,12817.41,1234.449951,622110000,1234.449951,12817.41,0.002093,0.007126,0.005194,...,1.130010e+09,1246.297994,13134.597000,120.847000,1.292925,13335.707833,1248.130042,1.189723e+09,1248.130042,13335.707833


In [77]:
InputData.to_csv("InputBaseData.csv") # CSV出力